<a href="https://colab.research.google.com/github/saurabhkirar/NLP_END/blob/main/Session8/Session8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Write a pyth0n program to multiply two number 
num1=100.19
num2=200.43
num3=num1*num2
print('The product of %.2f and %.2f is %.2f'%(num1,num2,num3))

The product of 100.19 and 200.43 is 20081.08


In [ ]:
# Write a program to print the positive number of all the  given number in a list 
lis=[1,3,-9,-81,7776,-9876]
abs_list=[]
print('actual list',lis)
for rec in lis:
  rec=abs(rec)
  abs_list.append(rec)
print('list with absolute values',abs_list)

actual list [1, 3, -9, -81, 7776, -9876]
list with absolute values [1, 3, 9, 81, 7776, 9876]


In [ ]:
## Write a program that has a python function to return the value if the input  is greater than zero else the function should return zero
# test the function with one postive and one negetive value.
def func_relu(x):
  return x if x > 0 else 0

print('the output for 98 is ',func_relu(98))
print('the output for -98  is ',func_relu(-98))

the output for 98 is  98
the output for -98  is  0


In [ ]:
## write a python code to find the maximum and mimimum values in a list.
lst=[98,776,1,22,33]
print('Maximum value in the list is ',max(lst))
print('minimum vale in the list is ',min(lst))

Maximum value in the list is  776
minimum vale in the list is  1


In [ ]:
## Write a function in python which takes a list as an input and returns the normalized list as the output.
## x normalized = (x – x minimum) / (x maximum – x minimum)
## test the function and print the normalized list
def func_normalization(p_list):
  l_max=max(p_list)
  l_min=min(p_list)
  deno=l_max-l_min
  norm_lst=[]
  for rec in p_list:
    norm_lst.append((rec-l_min)/deno)
  return norm_lst

nat_list=[234,345,887,98]
func_normalization(nat_list)

[0.17237008871989862, 0.31305449936628643, 1.0, 0.0]

In [ ]:
## Write a code to print the  memory address of a given array 
from array import *
arr=array('i',[1,5,7,89])
print('Memory address of the given array is ',arr.buffer_info())

Memory address of the given array is  (140549508420864, 4)


In [ ]:
## Write a code to print the  no of element of a given array 
from array import *
arr=array('i',[1,5,7,89])
print('There are total %d elements '%(arr.buffer_info()[1]))

There are total 4 elements 


In [ ]:
#Write a Python program to remove the first occurrence of a specified element from an array
from array import *
array_num = array('i', [1, 3, 15, 3, 37, 21, 29, 3])
print("Original array: "+str(array_num))
array_num.remove(3)
print("New array: "+str(array_num))

Original array: array('i', [1, 3, 15, 3, 37, 21, 29, 3])
New array: array('i', [1, 15, 3, 37, 21, 29, 3])


In [ ]:
## write a code to define dictionary and loop over it.
d = {'x': 10, 'y': 20, 'z': 30} 
for dict_key, dict_value in d.items():
    print(dict_key,'->',dict_value)

x -> 10
y -> 20
z -> 30


In [ ]:
## Write a code to merge two dictionaries.
d1 = {'a': 100, 'b': 200}
d2 = {'x': 300, 'y': 200}
d = d1.copy()
d.update(d2)
print(d)

{'a': 100, 'b': 200, 'x': 300, 'y': 200}


In [ ]:
## write a code to merge two lists into one dictionary.
keys = ['sachin', 'hayden', 'sana']
values = ['Ind','Aus', 'SL']
player = dict(zip(keys, values))
print(player)

{'sachin': 'Ind', 'hayden': 'Aus', 'sana': 'SL'}


In [ ]:
## write a class in oythin to get the dictionary from an object field, test the code.
class Player(object):
     def __init__(self):
         self.ind = 'sachin'
         self.aus = 'hayden'
         self.sl = 'sana'
     def do_nothing(self):
         pass
test = Player()
print(test.__dict__)

{'ind': 'sachin', 'aus': 'hayden', 'sl': 'sana'}


In [ ]:
"""
Write a Python program to print all unique values in a dictionary. Go to the editor
Sample Data : [{"V":"1"}, {"V": "2"}, {"VI": "1"}, {"VI": "5"}, {"VII":"5"}, {"V":"9"},{"VIII":"7"}]
Expected Output : Unique Values: {'5', '2', '7', '1', '9'}
"""
L = [{"V":"1"}, {"V": "2"}, {"VI": "1"}, {"VI": "5"}, {"VII":"5"}, {"V":"9"},{"VIII":"7"}]
print("Original List: ",L)
u_value = set( val for dic in L for val in dic.values())
print("Unique Values: ",u_value)

Original List:  [{'V': '1'}, {'V': '2'}, {'VI': '1'}, {'VI': '5'}, {'VII': '5'}, {'V': '9'}, {'VIII': '7'}]
Unique Values:  {'2', '1', '7', '9', '5'}


In [ ]:
## Write a pythin code to add three lists using lambda and map.
nums1 = [1, 2, 3] 
nums2 = [4, 5, 6] 
nums3 = [7, 8, 9] 
print("Original list: ")
print(nums1)  
print(nums2)  
print(nums3)  
result = map(lambda x, y, z: x + y + z, nums1, nums2, nums3) 
print("\nNew list after adding above three lists:")
print(list(result))

Original list: 
[1, 2, 3]
[4, 5, 6]
[7, 8, 9]

New list after adding above three lists:
[12, 15, 18]


In [ ]:
## use time and datetine and list down few formats 
import time
import datetime
print("Current date and time: " , datetime.datetime.now())
print("Current year: ", datetime.date.today().strftime("%Y"))
print("Month of year: ", datetime.date.today().strftime("%B"))
print("Week number of the year: ", datetime.date.today().strftime("%W"))
print("Weekday of the week: ", datetime.date.today().strftime("%w"))
print("Day of year: ", datetime.date.today().strftime("%j"))
print("Day of the month : ", datetime.date.today().strftime("%d"))
print("Day of week: ", datetime.date.today().strftime("%A"))

Current date and time:  2020-12-16 15:35:07.374382
Current year:  2020
Month of year:  December
Week number of the year:  50
Weekday of the week:  3
Day of year:  351
Day of the month :  16
Day of week:  Wednesday


In [12]:
## Write a code to show the unique file type based on the extension from a list.

lst=['a.jpg','b.png','c.jpeg','d.java','e.sql','f.sql','g.sql','z.png']
uniq=[]
for rec in lst:
    E =rec.split('.')[1]
    uniq.append(E)
set(uniq)  ## Set gives the distinct or non repetative values of the list.

{'java', 'jpeg', 'jpg', 'png', 'sql'}

In [13]:
## Write a python function to return the greatest common divisor (GCD) for two numbers, test the functions.
def gcd(x, y):
    gcd = 1
    
    if x % y == 0:
        return y
    
    for k in range(int(y / 2), 0, -1):
        if x % k == 0 and y % k == 0:
            gcd = k
            break  
    return gcd

print(gcd(12, 17))
print(gcd(4, 6))

1
2


In [14]:
## Write a code to calculate the euclidiean distance between two cordinates, express the cordinates  as list members.
import math
p1 = [4, 0]
p2 = [6, 6]
distance = math.sqrt( ((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2) )

print(distance)

6.324555320336759


In [15]:
## Write a code to see if any file exists in the location.
import os.path
open('abc.txt', 'w')
print(os.path.isfile('abc.txt'))

True


In [16]:
## Write a code to find out number of CPU using.
import multiprocessing
print(multiprocessing.cpu_count())

2


In [17]:
## Wite a code to find out profiling of python programs.
import cProfile
def res():
    print(100.72*344.87)
cProfile.run('res()')

34735.3064
         39 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-17-a8dede55eddf>:3(res)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:320(_schedule_flush)
        2    0.000    0.000    0.000    0.000 iostream.py:382(write)
        3    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        3    0.000    0.000    0.000    0.000 socket.py:432(send)
        3    0.000    0.000    0.000    0.000 threading.py:1062(_wait_for_tstate_lock)
        3    0.000    0.000    0.000    0.000 threading.py:1104(is_alive)
        3    0.000    0.000    0.000    0.000 threading.py:506(is_set)
        1    0.0

In [18]:
## Write a program to get the current user name 
import getpass
print(getpass.getuser())

root


In [1]:
## Write a code to print the URL content to the console.
from http.client import HTTPConnection
conn = HTTPConnection("google.com")
conn.request("GET", "/")  
result = conn.getresponse()
# retrieves the entire contents.  
contents = result.read() 
print(contents)

b'<HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">\n<TITLE>301 Moved</TITLE></HEAD><BODY>\n<H1>301 Moved</H1>\nThe document has moved\n<A HREF="http://www.google.com/">here</A>.\r\n</BODY></HTML>\r\n'


In [4]:
## write a pyhon function to give the running total and test with with a list.
from itertools import accumulate
import operator
def running_sum(it):
    return accumulate(it,operator.add)

#List
result = running_sum([1,2,3,4,5,6,7])
print("Running sum of a list:")
for i in result:
    print(i)

Running sum of a list:
1
3
6
10
15
21
28


In [5]:
#Write a Python program to check that a string contains only a certain set of characters (in this case a-z, A-Z and 0-9).
import re
def is_allowed_specific_char(string):
    charRe = re.compile(r'[^a-zA-Z0-9.]')
    string = charRe.search(string)
    return not bool(string)

print(is_allowed_specific_char("ABCDEFabcdef123450")) 
print(is_allowed_specific_char("*&%@#!}{"))

True
False


In [6]:
#Write a Python program that iterate over elements repeating each as many times as its count.
from collections import Counter
c = Counter(p=4, q=2, r=0, s=-2)
print(list(c.elements()))


['p', 'p', 'p', 'p', 'q', 'q']


In [8]:
## find the most common words in a sentence.
from collections import Counter
import re
text = """The start of T20 in a way revolutionised the cricket industry for good.  When we say this it is not just in terms of sports, fame, and viewership but also how it pumped up cricketer’s income. There are various leagues like BBL, IPL, CPL, PSL, and BPL which are pulling several big names across the globe which automatically helps in garnering massive revenue.

With the genesis of T20 cricket, cricket has witnessed revolutionary changes in terms of the popularity of the sport and the income earned by cricketers. The format has brought in a fast-moving, entertaining form of cricket to the world, forcing batsmen and bowlers to garner out-of-the-box solutions to counter the attacking prowess of the opponent.."""
words = re.findall('\w+',text)
print(Counter(words).most_common(10))

[('the', 10), ('of', 8), ('in', 5), ('cricket', 4), ('and', 4), ('to', 3), ('The', 2), ('T20', 2), ('a', 2), ('it', 2)]


In [10]:
#Write a Python program to find the most common elements and their counts of a specified text.
from collections import Counter
s = 'lkseropewdssafsdfafkpwe'
print("Original string: "+s)
print("Most common three characters of the said string:")
print(Counter(s).most_common(3))

Original string: lkseropewdssafsdfafkpwe
Most common three characters of the said string:
[('s', 4), ('e', 3), ('f', 3)]


In [11]:
## Write a code to merge two dictionaries 
import collections as ct
def merge_dictionaries(color1,color2):
    merged_dict = dict(ct.ChainMap({}, color1, color2))
    return merged_dict
color1 = { "R": "Red", "B": "Black", "P": "Pink" }
color2 = { "G": "Green", "W": "White" }
print("Original dictionaries:")
print(color1,' ',color2)
print("\nMerged dictionary:")
print(merge_dictionaries(color1, color2))

Original dictionaries:
{'R': 'Red', 'B': 'Black', 'P': 'Pink'}   {'G': 'Green', 'W': 'White'}

Merged dictionary:
{'W': 'White', 'G': 'Green', 'R': 'Red', 'P': 'Pink', 'B': 'Black'}


In [12]:
## write a code to print count of each element in the list.
sampleList = [11, 45, 8, 11, 23, 45, 23, 45, 89]
print("Original list ", sampleList)

countDict = dict()
for item in sampleList:
  if(item in countDict):
    countDict[item] += 1
  else:
    countDict[item] = 1
  
print("Printing count of each item  ",countDict)

Original list  [11, 45, 8, 11, 23, 45, 23, 45, 89]
Printing count of each item   {11: 2, 45: 3, 8: 1, 23: 2, 89: 1}


In [14]:
##Given a following two sets find the intersection and remove those elements from the first set
firstSet  = {23, 42, 65, 57, 78, 83, 29}
secondSet = {57, 83, 29, 67, 73, 43, 48}
print("First Set ", firstSet)
print("Second Set ", secondSet)

intersection = firstSet.intersection(secondSet)
print("Intersection is ", intersection)
for item in intersection:
  firstSet.remove(item)

print("First Set after removing common element ", firstSet)

First Set  {65, 42, 78, 83, 23, 57, 29}
Second Set  {67, 73, 43, 48, 83, 57, 29}
Intersection is  {57, 83, 29}
First Set after removing common element  {65, 42, 78, 23}


In [15]:
##Given two sets, Checks if One Set is Subset or superset of Another Set. if the subset is found delete all elements from that set
firstSet  = {57, 83, 29}
secondSet = {57, 83, 29, 67, 73, 43, 48}

print("First Set ", firstSet)
print("Second Set ", secondSet)

print("First set is subset of second set -", firstSet.issubset(secondSet))
print("Second set is subset of First set - ", secondSet.issubset(firstSet))

print("First set is Super set of second set - ", firstSet.issuperset(secondSet))
print("Second set is Super set of First set - ", secondSet.issuperset(firstSet))

if(firstSet.issubset(secondSet)):
  firstSet.clear()
  
if(secondSet.issubset(firstSet)):
  secondSet.clear()
print("First Set ", firstSet)
print("Second Set ", secondSet)

First Set  {57, 83, 29}
Second Set  {67, 73, 43, 48, 83, 57, 29}
First set is subset of second set - True
Second set is subset of First set -  False
First set is Super set of second set -  False
Second set is Super set of First set -  True
First Set  set()
Second Set  {67, 73, 43, 48, 83, 57, 29}
